In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation
import datetime
from datetime import timedelta as delta
from matplotlib import rc
from mpl_toolkits.basemap import Basemap

In [ ]:
from os import environ
environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

class ParticleData(object):
    def __init__(self):
        self.id = []

def load_particles_file(fname, varnames, s):
    T = ParticleData()
    pfile = Dataset(fname, 'r')
    T.id = pfile.variables['trajectory'][:]

    j = [k for k in range(pfile.variables['loc'].shape[0]) if pfile.variables['loc'][k, 0]==s]

    for v in varnames:
        setattr(T, v, pfile.variables[v][j, :].T)
    T.time_origin = datetime.datetime(1900, 12, 31)
    
    return T

Tall = {}
for s in range(1, 13):
    Tall[s] = load_particles_file('antarcticplastic_wstokes_7yr.nc', ['lon', 'lat', 'time'], s)

In [ ]:
plt.rcParams['figure.figsize'] = (8, 8)
fig = plt.figure()
m = Basemap(projection='spstere', boundinglat=-48,lon_0=180)
m.drawparallels(range(-90, 0, 10), labels=[False,False,False,False])
m.drawmeridians(range(0, 360, 15), labels=[True, True,True,True])
m.drawcoastlines(zorder=0)
m.fillcontinents(color='burlywood', zorder=0)

for i, s in enumerate(Tall):
    xs, ys = m(Tall[s].lon, Tall[s].lat)
    m.plot(xs[0], ys[0], '.', color=plt.cm.hsv(float(i)/len(Tall)), ms=10, label='Sampling Point #%d'%s)
    m.plot(xs[1:], ys[1:], '.', color=plt.cm.hsv(float(i)/len(Tall)), ms=2)

plt.legend(loc=1)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (14, 20)
fig = plt.figure()
for i, s in enumerate(Tall):
    ax = fig.add_subplot(4, 3, i+1)
    m = Basemap(projection='spstere', boundinglat=-50,lon_0=180)
    m.drawparallels(range(-90, 0, 10), labels=[False,False,False,False])
    m.drawmeridians(range(0, 360, 15), labels=[True,False,False,True])
    m.drawcoastlines(zorder=0)
    m.fillcontinents(color='burlywood', zorder=0)

    ax.set_title('Sampling Point #%d'%s)

    xs, ys = m(Tall[s].lon, Tall[s].lat)
    m.plot(xs, ys, '.', color=plt.cm.hsv(float(i)/len(Tall)), ms=2)

plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (7, 6)
def makemov(Tall):
    fig = plt.figure()
    frames = np.arange(0, 365*7, 14)
    
    def setmap():
        ax = fig.add_subplot(1, 1, 1)
        ttl = ax.set_title('Particles 0 years and   0 days before sampling')
        m = Basemap(projection='spstere', boundinglat=-48,lon_0=180)
        m.drawparallels(range(-90, 0, 10), labels=[False,False,False,False])
        m.drawmeridians(range(0, 360, 15), labels=[True,True,False,True])
        m.drawcoastlines(zorder=0)
        m.fillcontinents(color='burlywood', zorder=0)
        return m, ttl

    def setscat(T, m, col):
        xs, ys = m(T.lon[frames[0], :], T.lat[frames[0], :])
        scat = m.scatter(xs, ys, c=col, s=40, edgecolor='k', label='Sampling Point #%d'%s)
        return scat

    m, ttl = setmap()
    scat = {}
    for i, s in enumerate(Tall):
        scat[i] = setscat(Tall[s], m, plt.cm.hsv(float(i)/len(Tall)))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    def animate(t):
        for i, s in enumerate(Tall):
            if Tall[s].lon.shape[0] > t:
                xs, ys = m(Tall[s].lon[t, :], Tall[s].lat[t, :])
                scat[i].set_offsets(np.matrix((xs, ys)).transpose())
        ttl.set_text('Particles %d years and %2d weeks before sampling' % (t//365, (t % 365)/7))
        return scat

    rc('animation', html='html5')
    anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100, blit=False)
    
    plt.close()
    return anim
makemov(Tall)